In [1]:
import pandas as pd
import numpy as np


In [2]:
df_application_train = pd.read_csv('application_train.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'])
df_previous_application = pd.read_csv('previous_application.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'])
df_installment_payments = pd.read_csv('installments_payments.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'])
df_bureau = pd.read_csv('bureau.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'])
df_bureau_bal = pd.read_csv('bureau_balance.csv',header=0,error_bad_lines=False,index_col=0,nrows=5000)
df_pos_cash_bal = pd.read_csv('POS_CASH_balance.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'],nrows=5000)
df_credit_card_bal = pd.read_csv('credit_card_balance.csv',header=0,error_bad_lines=False,nrows=5000)

dflist = [df_application_train,df_previous_application,df_installment_payments,df_bureau,df_bureau_bal,df_pos_cash_bal,df_credit_card_bal]


/home/nervster/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df_previous_application = df_previous_application.drop(df_previous_application[df_previous_application['FLAG_LAST_APPL_PER_CONTRACT']=='N'].index)
df_previous_application = df_previous_application.drop(df_previous_application[(df_previous_application['AMT_APPLICATION']==0.0) & (df_previous_application['AMT_CREDIT']==0.0)].index)



In [4]:
df_previous_application[['RATE_DOWN_PAYMENT','RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED']] = df_previous_application[['RATE_DOWN_PAYMENT','RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED']].fillna(0.0)

In [5]:
df_prevapp_pivot = pd.pivot_table(df_previous_application,values=df_previous_application.columns,index='SK_ID_CURR',aggfunc={'SK_ID_PREV':(lambda x: len(x.unique())), 'NAME_CONTRACT_TYPE': (lambda x: len(x.unique())), 'AMT_ANNUITY':'sum', 'AMT_APPLICATION': 'sum',
       'AMT_CREDIT':'sum', 'AMT_DOWN_PAYMENT':'sum', 'AMT_GOODS_PRICE':'sum',
       'WEEKDAY_APPR_PROCESS_START':(lambda x: len(x.unique())), 'HOUR_APPR_PROCESS_START':np.mean,
       'FLAG_LAST_APPL_PER_CONTRACT':(lambda x: len(x.unique())), 'NFLAG_LAST_APPL_IN_DAY':(lambda x: len(x.unique())),
       'RATE_DOWN_PAYMENT':np.mean, 'RATE_INTEREST_PRIMARY':np.mean,
       'RATE_INTEREST_PRIVILEGED':np.mean, 'NAME_CASH_LOAN_PURPOSE':(lambda x: len(x.unique())),
       'NAME_CONTRACT_STATUS':(lambda x:x.map({'Approved':1,'Canceled':0,'Refused':0,'Unused offer':1}).mean()), 'DAYS_DECISION':np.mean, 'NAME_PAYMENT_TYPE':(lambda x: len(x.unique())),
       'CODE_REJECT_REASON':(lambda x: len(x.unique())), 'NAME_TYPE_SUITE':(lambda x: len(x.unique())), 'NAME_CLIENT_TYPE':(lambda x: len(x.unique())),
       'NAME_GOODS_CATEGORY':(lambda x: len(x.unique())), 'NAME_PORTFOLIO':(lambda x: len(x.unique())), 'NAME_PRODUCT_TYPE':(lambda x: len(x.unique())),
       'CHANNEL_TYPE':(lambda x: len(x.unique())), 'SELLERPLACE_AREA':(lambda x: len(x.unique())), 'NAME_SELLER_INDUSTRY':(lambda x: len(x.unique())),
       'CNT_PAYMENT':'sum', 'NAME_YIELD_GROUP':(lambda x: len(x.unique())), 'PRODUCT_COMBINATION':(lambda x: len(x.unique())),
       'DAYS_FIRST_DRAWING':'sum', 'DAYS_FIRST_DUE':'sum', 'DAYS_LAST_DUE_1ST_VERSION':'sum',
       'DAYS_LAST_DUE':'sum', 'DAYS_TERMINATION':'sum', 'NFLAG_INSURED_ON_APPROVAL':np.mean})

In [6]:
df_prevapp_pivot.head()

,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,CHANNEL_TYPE,CNT_PAYMENT,CODE_REJECT_REASON,DAYS_DECISION,DAYS_FIRST_DRAWING,...,NAME_YIELD_GROUP,NFLAG_INSURED_ON_APPROVAL,NFLAG_LAST_APPL_IN_DAY,PRODUCT_COMBINATION,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,SELLERPLACE_AREA,SK_ID_PREV,WEEKDAY_APPR_PROCESS_START
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,24835.5,23787.0,2520.0,24835.5,1,8.0,1,-1740.000000,365243.0,...,1,0.000000,1,1,0.104326,0.0,0.0,1,1,1
100002,9251.775,179055.0,179055.0,0.0,179055.0,1,24.0,1,-606.000000,365243.0,...,1,0.000000,1,1,0.000000,0.0,0.0,1,1,1
100003,169661.970,1306309.5,1452573.0,6885.0,1306309.5,3,30.0,1,-1305.000000,1095729.0,...,2,0.666667,1,3,0.033354,0.0,0.0,3,3,3
100004,5357.250,24282.0,20106.0,4860.0,24282.0,1,4.0,1,-815.000000,365243.0,...,1,0.000000,1,1,0.212008,0.0,0.0,1,1,1
100007,73672.830,903181.5,999832.5,6781.5,903181.5,4,124.0,1,-1222.833333,1826215.0,...,2,0.600000,1,4,0.053172,0.0,0.0,4,6,5


In [7]:
df = df_application_train.merge(df_prevapp_pivot,on =['SK_ID_CURR'],suffixes=('_app_train','_prev_app'))

In [8]:
df.head()

,TARGET,NAME_CONTRACT_TYPE_app_train,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_app_train,AMT_ANNUITY_app_train,AMT_GOODS_PRICE_app_train,...,NAME_YIELD_GROUP,NFLAG_INSURED_ON_APPROVAL,NFLAG_LAST_APPL_IN_DAY,PRODUCT_COMBINATION,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,SELLERPLACE_AREA,SK_ID_PREV,WEEKDAY_APPR_PROCESS_START_prev_app
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,1,0.000000,1,1,0.000000,0.0,0.0,1,1,1
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,2,0.666667,1,3,0.033354,0.0,0.0,3,3,3
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,1,0.000000,1,1,0.212008,0.0,0.0,1,1,1
100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,2,0.600000,1,4,0.053172,0.0,0.0,4,6,5
100009,0,Cash loans,F,Y,Y,1,171000.0,1560726.0,41301.0,1395000.0,...,4,0.000000,1,3,0.126602,0.0,0.0,4,7,4


In [9]:
df_bureau.columns

Index(['SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'DAYS_CREDIT',
       'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT',
       'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM',
       'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE',
       'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY'],
      dtype='object')

In [10]:
df_bureau_pivot = pd.pivot_table(df_bureau,values=df_bureau.columns,index='SK_ID_CURR',aggfunc={'SK_ID_BUREAU':'count','CREDIT_ACTIVE':'count','CREDIT_CURRENCY':(lambda x: len(x.unique())),'DAYS_CREDIT':np.min,'CREDIT_DAY_OVERDUE':'sum','DAYS_CREDIT_ENDDATE':'sum', 'DAYS_ENDDATE_FACT':'sum', 'AMT_CREDIT_MAX_OVERDUE':'sum', 'CNT_CREDIT_PROLONG':'sum', 'AMT_CREDIT_SUM':'sum','AMT_CREDIT_SUM_DEBT':'sum', 'AMT_CREDIT_SUM_LIMIT':'sum', 'AMT_CREDIT_SUM_OVERDUE':'sum',
       'CREDIT_TYPE':(lambda x: len(x.unique())), 'DAYS_CREDIT_UPDATE':np.min, 'AMT_ANNUITY':'sum'})

In [11]:
df.info


<bound method DataFrame.info of             TARGET NAME_CONTRACT_TYPE_app_train CODE_GENDER FLAG_OWN_CAR  \
SK_ID_CURR                                                                 
100002           1                   Cash loans           M            N   
100003           0                   Cash loans           F            N   
100004           0              Revolving loans           M            Y   
100007           0                   Cash loans           M            N   
100009           0                   Cash loans           F            Y   
100010           0                   Cash loans           M            Y   
100014           0                   Cash loans           F            N   
100015           0                   Cash loans           F            N   
100016           0                   Cash loans           F            N   
100017           0                   Cash loans           M            Y   
100018           0                   Cash loans         

In [12]:
df_installment_payments_pivot = pd.pivot_table(df_installment_payments,values=df_installment_payments.columns,index='SK_ID_CURR',aggfunc={'SK_ID_PREV':(lambda x: len(x.unique())),'NUM_INSTALMENT_VERSION':'sum', 'NUM_INSTALMENT_NUMBER':'sum','DAYS_INSTALMENT':'sum', 'DAYS_ENTRY_PAYMENT':'sum', 'AMT_INSTALMENT':'sum','AMT_PAYMENT':'sum'})                                                                                                                      

In [13]:
df_installment_payments_pivot['AMT_INSTAL_PAY_DIFF'] = df_installment_payments_pivot['AMT_INSTALMENT']-df_installment_payments_pivot['AMT_PAYMENT']
df_installment_payments_pivot['NUM_INSTAL_VERSION_NUM_DIFF'] = df_installment_payments_pivot['NUM_INSTALMENT_VERSION']-df_installment_payments_pivot['NUM_INSTALMENT_NUMBER']
df_installment_payments_pivot['DAY_INSTAL_ENTRY_DIFF'] = df_installment_payments_pivot['DAYS_INSTALMENT']-df_installment_payments_pivot['DAYS_ENTRY_PAYMENT']


In [14]:
df = df.merge(df_installment_payments_pivot,on =['SK_ID_CURR'],suffixes=('_curr2','_instpay'))

            TARGET NAME_CONTRACT_TYPE_app_train CODE_GENDER FLAG_OWN_CAR  \
SK_ID_CURR                                                                 
100002           1                   Cash loans           M            N   
100003           0                   Cash loans           F            N   
100004           0              Revolving loans           M            Y   
100007           0                   Cash loans           M            N   
100009           0                   Cash loans           F            Y   
100010           0                   Cash loans           M            Y   
100014           0                   Cash loans           F            N   
100015           0                   Cash loans           F            N   
100016           0                   Cash loans           F            N   
100017           0                   Cash loans           M            Y   
100018           0                   Cash loans           F            N   
100019      